**Data Preparation and Cleaning**

1. SETUP & IMPORTS

In [8]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
sns.set_theme(style="whitegrid")
pd.set_option('display.max_columns', None)

2. DATEN LADEN

In [14]:
current_dir = Path.cwd()
data_path = current_dir.parent / 'data' / 'OECD.WISE.WDP,DSD_HSL@DF_HSL_CWB,+all.csv'

print(f"Lade Daten von: {data_path}")

try:
    df = pd.read_csv(data_path)
    print("Daten erfolgreich geladen!")
except FileNotFoundError:
    print("FEHLER: Datei nicht gefunden. Prüfe den Pfad oder verschiebe die CSV in einen 'data' Ordner.")

Lade Daten von: /Users/Flurina/Library/CloudStorage/OneDrive-Persönlich/Documents/Uni_St.Gallen/BCS/3.Semester/Statistik_für_Data_Science/Projekt/Statistik_Projekt_OECDWellBeing/data/OECD.WISE.WDP,DSD_HSL@DF_HSL_CWB,+all.csv
Daten erfolgreich geladen!


3. ERSTE INSPEKTION

In [15]:
print(f"Dimensionen: {df.shape}")
print(df.head(3))
print(df.info())

Dimensionen: (108088, 30)
  STRUCTURE                           STRUCTURE_ID      STRUCTURE_NAME ACTION  \
0  DATAFLOW  OECD.WISE.WDP:DSD_HSL@DF_HSL_CWB(1.1)  Current well-being      I   
1  DATAFLOW  OECD.WISE.WDP:DSD_HSL@DF_HSL_CWB(1.1)  Current well-being      I   
2  DATAFLOW  OECD.WISE.WDP:DSD_HSL@DF_HSL_CWB(1.1)  Current well-being      I   

  REF_AREA Reference area MEASURE                Measure       UNIT_MEASURE  \
0      AUS      Australia    10_2  Feeling safe at night  PT_POP_Y_GE15_SUB   
1      AUS      Australia    10_2  Feeling safe at night  PT_POP_Y_GE15_SUB   
2      AUS      Australia    10_2  Feeling safe at night  PT_POP_Y_GE15_SUB   

                                     Unit of measure  AGE          Age SEX  \
0  Percentage of population aged 15 years or over...  MID  Middle-aged  _T   
1  Percentage of population aged 15 years or over...  MID  Middle-aged  _T   
2  Percentage of population aged 15 years or over...  MID  Middle-aged  _T   

     Sex EDUCATION_

4. DATEN BEREINIGUNG

In [17]:
cols_to_drop = [
    'STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'ACTION', 
    'REF_AREA',      # Wir haben 'Reference area'
    'MEASURE',       # Wir haben 'Measure'
    'UNIT_MEASURE',  # Wir haben 'Unit of measure'
    'AGE',           # Wir haben 'Age'
    'SEX',           # Wir haben 'Sex'
    'EDUCATION_LEV', # Wir haben 'Education level'
    'DOMAIN',        # Wir haben 'Domain'
    'OBS_STATUS', 'Observation status', 
    'UNIT_MULT', 'Unit multiplier', 
    'DECIMALS', 'Decimals', 
    'BASE_PER', 'Base period',
    'Observation value', # War leer, wir nutzen OBS_VALUE
    'Time period' # Ist oft leer oder doppelt, wir nutzen TIME_PERIOD
]

# Nur existierende Spalten droppen
df_clean = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')

# Spaltennamen normalisieren
df_clean.columns = [c.lower().replace(' ', '_') for c in df_clean.columns]
df_clean.rename(columns={'obs_value': 'value', 'time_period': 'year'}, inplace=True)

print("\n--- Spalten nach Bereinigung ---")
print(df_clean.columns.tolist())


--- Spalten nach Bereinigung ---
['reference_area', 'measure', 'unit_of_measure', 'age', 'sex', 'education_level', 'domain', 'year', 'value']


5. UMGANG MIT FILTERN & DUPLIKATEN

In [18]:
print("\nVerfügbare Measures (Top 10):")
print(df_clean['measure'].unique()[:10])


Verfügbare Measures (Top 10):
['Feeling safe at night' 'Not feeling safe at night'
 'Lack of social support' 'Social support' 'Feelings of physical pain'
 'Negative affect balance' 'Self-reported depression'
 'Average annual gross earnings'
 'Households and NPISHs net adjusted disposable income per capita'
 'Median net wealth']


In [22]:
# Sortieren nach Jahr, damit 'last' wirklich das aktuellste ist
df_clean = df_clean.sort_values('year')

#Einheiten definieren
group_cols = ['reference_area', 'measure', 'sex', 'age', 'education_level', 'domain']

df_time = df_clean.copy()

#Filtern auf 1 Jahr, um Unabhängigkeit zu wahren
df_latest = df_clean.drop_duplicates(subset=group_cols, keep='last')

print(f"Datensatz für Zeitreihen (df_time): {len(df_time)} Zeilen")
print(f"Datensatz für Hypothesentests (df_latest): {len(df_latest)} Zeilen")

Datensatz für Zeitreihen (df_time): 108088 Zeilen
Datensatz für Hypothesentests (df_latest): 10870 Zeilen


6. MISSING VALUES HANDLING

In [23]:
# Beide Datensätze von leeren Werten in der Zielvariable bereinigen
df_time = df_time.dropna(subset=['value'])
df_latest = df_latest.dropna(subset=['value'])



7. SPEICHERN

In [24]:
df_time.to_csv(current_dir.parent / 'data' / 'oecd_full_time_series.csv', index=False)
df_latest.to_csv(current_dir.parent / 'data' / 'oecd_snapshot_latest.csv', index=False)

print("Beide Datensätze gespeichert.")

Beide Datensätze gespeichert.


Für die Analyse haben wir **zwei Versionen des Datensatzes** erstellt:

**oecd_full_time_series.csv** : Enthält alle historischen Daten. Wird verwendet für deskriptive Zeitreihenanalysen (Trendentwicklung).

**oecd_snapshot_latest.csv** : Enthält nur den jeweils aktuellsten Datenpunkt pro Land und Kategorie. Wird verwendet für induktive Statistik (Hypothesentests, Korrelationen), um die statistische Annahme der Unabhängigkeit der Beobachtungen zu wahren und Pseudoreplikation durch wiederholte Messungen desselben Landes zu vermeiden.